In [4]:
import camelot
import pandas
import os
import copy
from datetime import datetime, timedelta
import feedparser
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
# from twitter_post import post
import re
import PyPDF2
from pathlib import Path

In [12]:
pdf_file_path = Path.cwd().parent.joinpath("data", "202105.pdf")

print(f"generating {pdf_file_path}")


pdf_read = PyPDF2.PdfFileReader(pdf_file_path.__str__())
num_page = pdf_read.numPages

if num_page > 10:
    page_list = list(range(10, num_page, 10))
else:
    page_list = [num_page]
init = 1
dfs_lap = []
for num in page_list:
    if num == page_list[-1]:
        num = "end"
    tbls = camelot.read_pdf(pdf_file_path.__str__(), pages=f'{init}-{num}')
    if num != "end":
        init = num + 1

    dfs = []
    for table in tbls:
        df = table.df
        dfs.append(df)
    df_all = pandas.concat(dfs)
    dfs_lap.append(df_all)
df_total = pandas.concat(dfs_lap)

generating C:\Users\shuny\Documents\twitter_aichi_covid19\data\202105.pdf


In [15]:
df_total.to_pickle("database.zip")

In [17]:


df_all = copy.deepcopy(df_total)

# NoをIndexにする
df_all.columns = df_all.iloc[0, :]
df_all = df_all.sort_values("No")
df_all = df_all[df_all["No"] != "No"]
df_all = df_all.set_index("No")

In [78]:
# 発表日が欠番なものを削除
df_all = df_all[~df_all["発表日"].str.contains("欠番")]

# インデックスが修正なものを削除
df_all = df_all[~df_all.index.str.contains("修正")]


# 発表日のコラムに\n区切りで他のデータが紛れ込んでいるものを修正
issued_date = [_.split("\n") for _ in df_all["発表日"].to_list()]
refined_list = []
for i, _ in enumerate(issued_date):
    if len(_) > 1:
        
        new_list = _+ [elem for elem in df_all.iloc[i, :].to_list() if (elem!="")&(elem!="\n".join(_))]
        new_list.insert(2, "")
        if len(new_list) == 5:
            new_list.insert(4, "")
        refined_list.append(new_list)
    else:
        refined_list.append(df_all.iloc[i, :].to_list())

for i in range(len(df_all.index)):
    df_all.iloc[i,:] = refined_list[i]


In [106]:
# インデックスの中に発表日が紛れ込んでいるものを修正
new_index = []
new_date = []
for i, _ in enumerate(df_all.index.to_list()):
    if not _.isdigit():
        index, remaining = _.split()
        remaining = remaining.replace("※", "")
        new_index.append(int(index))
        new_date.append(remaining)
    else:
        new_index.append(int(_))
        new_date.append(df_all.iloc[i, :]["発表日"])
df_all.index = new_index
df_all["発表日"] = new_date

In [108]:
# インデックスでソート
df_all = df_all.sort_index()

In [115]:
# 2020年と2021年のはざまを検索する
months = [int(_.split("月")[0]) for _ in df_all["発表日"].to_list()]
for i, month in enumerate(months): 
    if month > months[i+1]+10:
        year_start = i
        break

# 発表日をstrからdatetimeに修正
df_all["発表日"] = pandas.to_datetime(
    ['2021年' + _ if no > year_start else '2020年' + _ for no, _ in enumerate(df_all['発表日'])], format='%Y年%m月%d日')

In [117]:
df_all.to_pickle("202105.zip")